<a href="https://colab.research.google.com/github/mswastik/optimization/blob/master/CP4.1%20cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Indices**  
j - job  
m - machine 

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  
$b_{j,m}$ - machine m is selected for job j     
$a_{j,i,m}$ - job i precedes job j at machine m 

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  
$set_{i,j}$ - setup time between job i & j   
$C_d $ - Order delay cost    
$C_i$ - Inventory holding cost    
$C_o$ - Changeover cost    



**Objective** (Not implemented right now)  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$    


**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $    
$ \sum_m s_{i,m} >= e_{j,m} + set_{i,j} \qquad if \quad b_{j,m}=1, b_{i,m}=1, a_{j,i,m}=1 $  

In [2]:
!pip install ortools
#!pip install dtale
#!pip install -U plotly

In [3]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo


In [4]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='Sheet2')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')
 
# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
df.drop(['Dispatch Date','SO Date','Description','Customer'],axis=1,inplace=True)
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']
 
 
# Creating Parameter Data
jobs =df['Key'].copy()
op =[1,2]  # Ignore as of now
mc = [1,2,3,4,5,6]
#due = df.set_index('Key')
df1=df.copy()
#df1 = df1.set_index('Key')
#dur=df[['Key','FG Code','Order Qty']]
df1=df1.merge(pa[["FG Code","Line","Output"]], on="FG Code")
df1['Output']=df1['Output']*100
df1['duration']=df1['Order Qty']/df1['Output']
df1['duration'] = df1['duration'].astype('float64').round(decimals = 2)
df1['operation'] = 2
budf=df.merge(ma[["FG Code","Bulk Code","Case Size","Pack Wt(g)"]],on='FG Code',how='left')
budf['Qty']=budf['Order Qty']*budf['Case Size']*budf['Pack Wt(g)']/1000
 
pasp= pa.groupby('FG Code').mean()
budf=budf.merge(pasp,on='FG Code')
budf['cons'] = budf['Speed']*budf['Pack Wt(g)']*60/1000
budf.drop(['Speed','Line'],axis=1,inplace=True)
budf=budf.merge(bu[["Bulk Code", "Line","Output"]],on="Bulk Code",how="left")
 
budf['Output'] = budf['Output']*100
 
budf['Output'] = budf['Output'].replace(0,1)
budf['duration'] = budf['Qty']/budf['Output']
budf['duration'] = budf['duration'].fillna(0).astype('int')
budf['operation'] = 1
budf['Order Qty'] = budf['Qty']
#budf.drop(['Plant','due','sodate','Case Size','Pack Wt(g)','SO','Qty','Bulk Code'],axis=1,inplace=True)
budf.drop('Qty',axis=1,inplace=True)
 
df1=df1.append(budf)
df1.set_index(['Key','Line','operation'],inplace=True)
#del df1['FG Code']
df1['Output'].replace(0,1,inplace=True)
df1['Order Qty']=df1['Order Qty'].fillna(0).astype('int')
df1['duration']=df1['duration'].fillna(0).astype('int')
df1['Output']=df1['Output'].fillna(0).astype('int')
#horizon = sum(df1['duration'])
horizon = max(df["due"])+300
 
#Setup
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[40]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)
 
Cd = 125
Ci = 110
Cc = 12000

In [83]:
# Initialise model
model = cp_model.CpModel()
 
# Creating variables to store data
task_type = collections.namedtuple('task_type', 'start end dur run1')
mtj = collections.defaultdict(list)
all_tasks = {}
 
for j in jobs:
  for o in op:
    for m in mc:
      suffix = '_%s_%i_%i' % (j,m,o)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      run1 = model.NewIntVar(0, df1['duration'].loc[(j,m,o)].item(), "run"+suffix)
      duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
      all_tasks[j, m,o] = task_type(start=start, end=end, dur= duration, run1=run1)
      mtj[o,m].append(duration)
 
for j in jobs:
  model.Add(all_tasks[j, 5,1].run1==0)
  model.Add(all_tasks[j, 6,1].run1==0)
 
# Selection variable to store whether machine is selected to process job
b={}
for o in op:
  for j in jobs:
    for m in mc:
      b[j,m,o] = model.NewBoolVar('selection')
      model.Add(all_tasks[j, m,o].run1>0).OnlyEnforceIf(b[j,m,o])
      model.Add(all_tasks[j, m,o].run1==0).OnlyEnforceIf(b[j,m,o].Not())

# (New) Small run
for o in op:
  for m in mc:
    for j in jobs:
      if df1['duration'].loc[(j,m,o)].item()<120:
        model.Add(all_tasks[j, m,o].run1==df1['duration'].loc[(j,m,o)].item()).OnlyEnforceIf(b[j,m,o])
        model.Add(all_tasks[j, m,o].run1==0).OnlyEnforceIf(b[j,m,o].Not())

# (New) Paste Transfer
for j in jobs:
  model.Add(all_tasks[j, 5,2].run1==0).OnlyEnforceIf(b[j,1,1])
  model.Add(all_tasks[j, 6,2].run1==0).OnlyEnforceIf(b[j,1,1])
  model.Add(all_tasks[j, 6,2].run1==0).OnlyEnforceIf(b[j,2,1])
  model.Add(all_tasks[j, 5,2].run1==0).OnlyEnforceIf(b[j,2,1])
  model.Add(all_tasks[j, 1,2].run1==0).OnlyEnforceIf(b[j,3,1])
  model.Add(all_tasks[j, 2,2].run1==0).OnlyEnforceIf(b[j,3,1])
  model.Add(all_tasks[j, 3,2].run1==0).OnlyEnforceIf(b[j,3,1])
  model.Add(all_tasks[j, 1,2].run1==0).OnlyEnforceIf(b[j,4,1])
  model.Add(all_tasks[j, 2,2].run1==0).OnlyEnforceIf(b[j,4,1])
  model.Add(all_tasks[j, 3,2].run1==0).OnlyEnforceIf(b[j,4,1])
  model.Add(all_tasks[j, 4,2].run1==0).OnlyEnforceIf(b[j,4,1])

# Variable to store sequence of jobs
a={}
for o in op:
  for j in jobs:
    for m in mc:
      for i in jobs:
        a[j,i,m,o] = model.NewBoolVar('sequence')
        if j==i:
          model.Add(all_tasks[i, m,o].start==all_tasks[j, m,o].start)
        if j != i and b[j,m,o] and b[i,m,o]:
          model.Add(all_tasks[i, m,o].start>all_tasks[j, m,o].start).OnlyEnforceIf(a[j,i,m,o])
          model.Add(all_tasks[i, m,o].start<all_tasks[j, m,o].start).OnlyEnforceIf(a[j,i,m,o].Not())
 
# Constraint: A machine can process only 1 job at a time 
for o in op:
  for m in mc:
    model.AddNoOverlap(mtj[o,m])
 
# Constraint: Complete production of full order quantity
com = {}
for o in op:
  for j in jobs:
    tt= 0
    for m in mc:
      comp = all_tasks[j,m,o].run1*df1["Output"].loc[(j,m,o)]
      tt = tt + comp
    com[j] = tt
    model.Add(com[j]*10 >= df1.loc[(j,slice(None),o)]['Order Qty'].values[0])
  
# Constraint of Setup Time
for o in op:     
  for j in jobs:
    for m in mc:
      for i in jobs:
        model.Add(all_tasks[i, m,o].start >= all_tasks[j, m,o].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[j,i,m,o]).OnlyEnforceIf(b[j,m,o]).OnlyEnforceIf(b[i,m,o])
'''       
# Delay between jobs due to storage tank full
for j in jobs:
  for m in mc:
    for i in jobs:
      if df1.loc[(j,m,1)]['Order Qty']>=3000:
        model.Add(all_tasks[i, m,1].start >= all_tasks[j, m,1].end+int((df1.loc[(j,m,1)]['Order Qty']-3000)/df1.loc[(j,m,1)]['cons'])).OnlyEnforceIf(a[j,i,m,o])
        #model.AddReservoirConstraint()
'''
for j in jobs:
  for m in mc:
    minst=model.NewIntVar(0, horizon, "minst"+j)
    model.AddMinEquality(minst,[all_tasks[j,k,1].start for k in mc])
    model.Add(all_tasks[j, m,2].start >= minst+4).OnlyEnforceIf(b[j,m,2])
 
for j in jobs:
  for m in mc:
    minst=model.NewIntVar(0, horizon, "minst"+j)
    model.AddMinEquality(minst,[all_tasks[j,k,1].start for k in mc])
    model.Add(all_tasks[j, m,2].start >= minst+4).OnlyEnforceIf(b[j,m,2])
    model.Add(all_tasks[j, m,2].start <= minst+24).OnlyEnforceIf(b[j,m,2])
 
d={}
for o in op:
  for j in jobs:
    for m in mc:
      d[j,m,o] = model.NewBoolVar('delay')
      model.Add(all_tasks[j,m,o].end <= df1['due'].loc[j].values[0]).OnlyEnforceIf(d[j,m,o])
      model.Add(all_tasks[j,m,o].end > df1['due'].loc[j].values[0]).OnlyEnforceIf(d[j,m,o].Not())
 
tc=0
tt1={}
s1={}
s3={}
expr={}
setcost=0
for i in jobs:
  for m in mc:
    for j in jobs:
      setcost += a[j,i,m,o]
      if setcost==1:
        setcost = Cc
    suffix = '_%s_%i' % (i,m)
    s1[i,m] = model.NewIntVar(-Cd*horizon, Cd*horizon, 's1'+suffix)
    model.Add(s1[i,m] == Ci * (df1['due'].loc[i].values[0]-48 - all_tasks[i,m,2].end))
    # Second segment.
    #s2[i,m] = 0
    # Third segment.
    s3[i,m] = model.NewIntVar(-Cd*horizon, Cd*horizon, 's3'+suffix)
    model.Add(s3[i,m] == Cd * (all_tasks[i,m,2].end - df1['due'].loc[i].values[0]-48))
    # Link together expr and x through s1, s2, and s3.
    #expr[i,m]=0
    expr[i,m] = model.NewIntVar(-Cd *horizon, Cd*horizon, 'expr'+suffix)
    model.AddMaxEquality(expr[i,m], [s1[i,m], 0, s3[i,m]])
    '''
    if all_tasks[i,m,2].end > df1['due'].loc[i].values[0]+48:
      delcost = Cd
    if all_tasks[i,m,2].end < df1['due'].loc[i].values[0]-48:
      earcost = Ci
    '''
    #tt1[i,m] = sum(expr.values())
 
#tc = sum(tt1.values())
 
model.Minimize(sum(expr.values()))
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 16
solver.Solve(model)
print(solver.StatusName(),solver.ObjectiveValue())

FEASIBLE 100375.0


In [119]:
ff=pd.DataFrame()
for o in op:
  for m in mc:
    for j in jobs:
      if solver.Value(all_tasks[j,m,o].run1)>0:
        kk={'Job':j,'line':m,'operation':o,'start':solver.Value(all_tasks[j,m,o].start),
            'run':solver.Value(all_tasks[j,m,o].run1),'end':solver.Value(all_tasks[j,m,o].end),'due':df1['due'].loc[j].values[0]}
        ff=ff.append(kk,ignore_index=True)

!pip install -U plotly
import plotly.express as px
import datetime
#fig=px.timeline(data_frame=ff, x_start=ff['start'].astype('datetime64[h]'),x_end=ff['end'].astype('datetime64[h]'),
#                facet_row='line',facet_col='operation',y='Job',height=1200,width=1500,color='Job')
fig=px.timeline(data_frame=ff, x_start=ff['start'].astype('datetime64[h]'),x_end=ff['end'].astype('datetime64[h]'),y=ff['line'].astype('str'),facet_col='operation',color=ff["Job"])
fig.update_xaxes(dtick=14400000,tickformat="%H-%d")
fig.update_yaxes(autorange="reversed")


Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.14.1)


In [138]:
from bokeh.io import output_notebook, show
import bokeh.plotting
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20 as palette

output_notebook()
tools = ["box_select", "hover", "reset"]
#x = [ (fruit, year) for fruit in fruits for year in years ]
#sprint.Year = sprint.Year.astype(str)
#group = ff.groupby(['line']).sum()
length = len(ff)
#colors = palette[length]
#source = ColumnDataSource(ff)
#while len(colors)>length:
#    colors.pop()
ff['line'] = ff['line'].astype('str')
ff['operation'] = ff['operation'].astype('str')
kkk=ff['line'].unique()
ff1=ff.set_index(['operation','Job','line'])
#group = ff.groupby(by=['Job','operation','line'])
source = ColumnDataSource(ff1)

p = figure(title="Schedule", tools=tools,tooltips=[("Operation:","@operation"), ("Start:","@start"), ("End:","@end")])
#p.hbar(y='operation_Job_line', left='start_min', right='end_min', height=0.4,source=source,color=factor_cmap('line', Spectral6, sorted(kkk)))
p.hbar(y='operation_Job_line', left='start', right='end',source=source,legend='operation_Job_line')
#p.yaxis.group_label_orientation="horizontal"
#p.yaxis.subgroup_label_orientation="horizontal"

r = figure(title="Schedule", tools=tools,tooltips=[("Job:","@Job"), ("Start:","@start"), ("End:","@end")])
r.hbar(y='Job', left='start', right='end',source=source,legend_label='Job')

pg = gridplot([[p],[r]])
show(pg)

/usr/local/lib/python3.6/dist-packages/bokeh/util/deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead



In [135]:
source.data

{'due': array([80., 87., 87., 93., 93., 82., 82., 82., 87., 87., 87., 88., 80.,
        82., 88., 93., 80., 87., 88., 87., 87., 88., 82., 87., 93., 93.,
        82., 87., 80., 82., 87., 80., 88., 80., 87., 93.]),
 'end': array([165.,  64., 322., 379., 229., 113., 329., 226.,  56., 170., 273.,
        385., 383., 228., 269., 132., 260., 342., 104., 215.,  36., 100.,
         23.,  77., 132., 372., 207.,  97.,  21., 308., 161.,  49., 134.,
        303.,  79., 155.]),
 'operation_Job_line': array([('1.0', '17023287gd29904300iX', '1.0'),
        ('1.0', '17023335gd30212200iX', '1.0'),
        ('1.0', '17023637gd28002400iX', '1.0'),
        ('1.0', '4501763335gd23010000Ud', '1.0'),
        ('1.0', '4501763335gC25020000UC', '1.0'),
        ('1.0', '17023572gd43208500iX', '2.0'),
        ('1.0', '17023636gd280043C0iX', '2.0'),
        ('1.0', '17023636gd28001701iX', '2.0'),
        ('1.0', '17023335gd28018100iX', '2.0'),
        ('1.0', '17023335gd28001700iX', '2.0'),
        ('1.0', '1702363

In [ ]:
!pip install dtale
from dtale import show
show(df1,ignore_duplicate=True)

In [ ]:
#Exploring Result (click on last link to view result)
#!pip install dtale
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
#show(dur,ignore_duplicate=True)

In [ ]:
show(df,ignore_duplicate=True)

https://toi3k07s9do-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/2

In [ ]:
print(model.ModelStats())

Optimization model '':
#Variables: 2040 (17 in objective)
 - 1734 in [0,1]
 - 5 in [0,15]
 - 5 in [0,36]
 - 1 in [0,66]
 - 5 in [0,75]
 - 5 in [0,93]
 - 5 in [0,108]
 - 5 in [0,133]
 - 1 in [0,162]
 - 2 in [0,168]
 - 3 in [0,189]
 - 1 in [0,213]
 - 2 in [0,216]
 - 5 in [0,233]
 - 3 in [0,243]
 - 1 in [0,300]
 - 2 in [0,320]
 - 2 in [0,400]
 - 8 in [0,450]
 - 3 in [0,451]
 - 1 in [0,462]
 - 1 in [0,497]
 - 2 in [0,693]
 - 2 in [0,746]
 - 1 in [0,940]
 - 3 in [0,978]
 - 1 in [0,995]
 - 3 in [0,1054]
 - 6 in [0,1066]
 - 2 in [0,1493]
 - 5 in [0,2000]
 - 3 in [0,2108]
 - 1 in [0,2160]
 - 1 in [0,4500]
 - 1 in [0,5600]
 - 1 in [0,8000]
 - 1 in [0,8640]
 - 1 in [0,14000]
 - 1 in [0,15120]
 - 1 in [0,36000]
 - 204 in [0,1442570]
#kInterval: 102
#kLinear1: 204 (#enforced: 204)
#kLinear2: 4896 (#enforced: 4896)
#kLinearN: 17
#kNoOverlap: 6
